In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [4]:
mpa = pd.read_csv('Data/Outputs/mushed.csv')

In [5]:
flights = pd.read_csv('Data/Outputs/flights.csv')

In [8]:
mpa.drop('country', inplace=True, axis=1)

In [17]:
mpa[mpa['category']=='Airport']

,Unnamed: 0,iata,airport,city,state,latitude,longitude,category,id,name,neighbourhood,room_type,price,minimum_nights,availability_365,Labels,q_price
0,0,ABQ,Albuquerque International,Albuquerque,NM,35.040222,-106.609194,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,8,79.0
1,1,ANC,Ted Stevens Anchorage International,Anchorage,AK,61.174320,-149.996186,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,32,106.0
2,2,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,16,80.0
3,3,AUS,Austin-Bergstrom International,Austin,TX,30.194533,-97.669872,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,4,64.0
4,4,BDL,Bradley International,Windsor Locks,CT,41.938874,-72.683228,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,26,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,336,WRG,Wrangell,Wrangell,AK,56.484326,-132.369824,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,28,75.0
337,337,WYS,Yellowstone,West Yellowstone,MT,44.688399,-111.117638,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,29,100.0
338,338,XNA,Northwest Arkansas Regional,Fayetteville Springdale Rogers,AR,36.281869,-94.306811,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,33,100.0
339,339,YAK,Yakutat,Yakutat,AK,59.503361,-139.660226,Airport,43806155.0,Home away from home,Unincorporated Areas,Entire home/apt,100,1.0,0.0,28,75.0


In [ ]:
Pseudocode:
    search_func(from and to)
    get label of from
    get label of to
    get all in label of to
    get all in label of from
    get all combinations of all_from and all_to
    reorder all combinatins so all_from are indexed first
    